In [1]:
import os
from datetime import datetime, timedelta

import numpy as np
import matplotlib.pyplot as plt
import cmocean.cm as cmo
from matplotlib.gridspec import GridSpec
from cartopy.crs import LambertAzimuthalEqualArea
from cartopy.feature import LAND

srs_dst = LambertAzimuthalEqualArea(central_longitude=0, central_latitude=90)
%matplotlib inline

In [ ]:
date0 = datetime(2018, 1, 1)
c_sum0 = []
c_sum1 = []
maps0 = []
maps1 = []
map_dates = []

no_years = 3
step = 3

for i in range(0, 366*no_years, step):
    idate = date0 + timedelta(i)
    msh_file0 = f'../NERSC_arctic25km_sea_ice_age_v2p1/mesh/{idate.year}/mesh_{idate.strftime("%Y%m%d")}.npz'
    myi_file0 = f'../NERSC_arctic25km_sea_ice_age_v2p1/sic/2017/sic_20170915_{idate.strftime("%Y%m%d")}.npz'

    msh_file1 = f'../NERSC_arctic25km_sea_ice_age_v2p1/overlap/mesh/{idate.year}/mesh_{idate.strftime("%Y%m%d")}.npz'
    myi_file1 = f'../NERSC_arctic25km_sea_ice_age_v2p1/overlap/sic/2017/sic_20170915_{idate.strftime("%Y%m%d")}.npz'
    if not os.path.exists(myi_file1):
        break

    with np.load(msh_file0) as f:
        x0 = f['x'] * 1000
        y0 = f['y'] * 1000
        t0 = f['t']
    c0 = np.load(myi_file0)['c']    

    with np.load(msh_file1) as f:
        x1 = f['x'] * 1000
        y1 = f['y'] * 1000
        t1 = f['t']
    c1 = np.load(myi_file1)['c']    

    c_sum0.append(np.sum(c0))
    c_sum1.append(np.sum(c1))

    if i in np.linspace(0, 360*no_years, 6).astype(int):
        maps0.append([x0,y0,t0,c0])
        maps1.append([x1,y1,t1,c1])
        map_dates.append(idate)
        print(f'Added map for {idate.strftime("%Y%m%d")}')

In [ ]:
fig = plt.figure(figsize=(9, 6))

plt.clf()
gs = GridSpec(3, len(maps0), figure=fig)
for i, (map0, map1, map_date) in enumerate(zip(maps0, maps1, map_dates)):
    x0, y0, t0, c0 = map0
    ax = fig.add_subplot(gs[0, i], projection=srs_dst)
    ax.tripcolor(x0, y0, t0, c0, clim=[0,100], cmap=cmo.ice)
    ax.set_title(f'{map_date.strftime("%Y-%m-%d")}')
    ax.add_feature(LAND, facecolor='lightgray', edgecolor='black', zorder=1)
    ax.set_xlim([-1800000, 2000000])
    ax.set_ylim([-2000000, 2000000])

    x1, y1, t1, c1 = map1
    ax = fig.add_subplot(gs[1, i], projection=srs_dst)
    ax.tripcolor(x1, y1, t1, c1, clim=[0,100], cmap=cmo.ice)

    ax.add_feature(LAND, facecolor='lightgray', edgecolor='black', zorder=1)
    ax.set_xlim([-1800000, 2000000])
    ax.set_ylim([-2000000, 2000000])

# Third row: area time series
days = np.arange(0, len(c_sum0)) * step
ax = fig.add_subplot(gs[2, :])
ax.plot(days, c_sum0, label='SID CDR')
ax.plot(days, c_sum1, label='SID NRT')
ax.plot(days, np.abs(np.array(c_sum1) - np.array(c_sum0)), label='| NRT - CDR |')
ax.fill_between([151, 242], 0, 1.2e6, color='lightgray', alpha=0.5, label='summer')
ax.fill_between([516, 607], 0, 1.2e6, color='lightgray', alpha=0.5)
ax.fill_between([882, 973], 0, 1.2e6, color='lightgray', alpha=0.5)
ax.set_title('Area of advected MYI (m$^2$)')
ax.set_xlabel('Days since 2018-01-01')
ax.set_ylim([0, 1.2e6])
ax.legend()
plt.tight_layout()
plt.savefig('figure14_myi_area_comparison.png', dpi=100, bbox_inches='tight', pad_inches=0.1)
plt.show()

In [ ]:
(datetime(2018,6,1) - datetime(2018,1,1),
 datetime(2018,8,31) - datetime(2018,1,1),
 datetime(2019,6,1) - datetime(2018,1,1),
 datetime(2019,8,31) - datetime(2018,1,1), 
 datetime(2020,6,1) - datetime(2018,1,1),
 datetime(2020,8,31) - datetime(2018,1,1), 
)

In [ ]:
dts_date = '20201218'
cfile00 = f'../NERSC_arctic25km_sea_ice_age_v2p1/sic/2017/sic_20170915_{dts_date}.npz'
mfile00 = f'../NERSC_arctic25km_sea_ice_age_v2p1/mesh/2020/mesh_{dts_date}.npz'

cfile0 = f'../NERSC_arctic25km_sea_ice_age_v2p1/overlap/sic/2017/sic_20170915_{dts_date}.npz'
mfile0 = f'../NERSC_arctic25km_sea_ice_age_v2p1/overlap/mesh/2020/mesh_{dts_date}.npz'

cfile1 = f'../NERSC_arctic25km_sea_ice_age_v2p1/overlap/sic/2017/new_remeshing/sic_20170915_{dts_date}.npz'
mfile1 = f'../NERSC_arctic25km_sea_ice_age_v2p1/overlap/mesh/2020/new_remeshing/mesh_{dts_date}.npz'

c00 = np.load(cfile00)['c']
c0 = np.load(cfile0)['c']
c1 = np.load(cfile1)['c']

with np.load(mfile00) as f:
    x00 = f['x'] * 1000
    y00 = f['y'] * 1000
    t00 = f['t']

with np.load(mfile0) as f:
    x0 = f['x'] * 1000
    y0 = f['y'] * 1000
    t0 = f['t']

with np.load(mfile1) as f:
    x1 = f['x'] * 1000
    y1 = f['y'] * 1000
    t1 = f['t']

fig, axs = plt.subplots(1, 3, figsize=(12, 8), sharex=True)
axs[0].tripcolor(x00, y00, t00, c00, clim=[0, 100], cmap=cmo.ice)
axs[1].tripcolor(x0, y0, t0, c0, clim=[0, 100], cmap=cmo.ice)
axs[2].tripcolor(x1, y1, t1, c1, clim=[0, 100], cmap=cmo.ice)

for ax in axs:
    ax.set_aspect('equal')